# import libraries

In [193]:
import pandas as pd

# file operations

In [194]:
def read_csv_file(path):
    df = pd.read_csv(path)
    return df

# Read Data

In [195]:
df = read_csv_file('result_emotion.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 87 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             400 non-null    int64  
 1   text                   400 non-null    object 
 2   label                  400 non-null    int64  
 3   anger_result           400 non-null    float64
 4   anger_prompt0_meta     400 non-null    object 
 5   anger_prompt0          400 non-null    int64  
 6   anger_prompt1_meta     400 non-null    object 
 7   anger_prompt1          400 non-null    int64  
 8   anger_prompt2_meta     400 non-null    object 
 9   anger_prompt2          400 non-null    int64  
 10  anger_prompt3_meta     400 non-null    object 
 11  anger_prompt3          400 non-null    int64  
 12  anger_prompt4_meta     400 non-null    object 
 13  anger_prompt4          400 non-null    int64  
 14  anger_prompt5_meta     400 non-null    object 
 15  anger_

In [196]:
key ="joy"
prompt_num = 10
golden_label = 1

for i in range(prompt_num):
    df["{key}_prompt{id}".format(key=key, id=i)] = df["{key}_prompt{id}_meta".format(key=key, id=i)].apply(
        lambda x: 1 
            if x.find("My Answer is Yes") != -1 
            else 0 if x.find("My Answer is No") != -1
            else 0
    )

# Analyze Data

In [197]:
def analyze(df, threshold, key, golden_label):
    df["{}_hypothesis".format(key)] = df["{}_result".format(key)].apply(lambda x: 1 if x >= threshold else 0)

    count_true_positive = len(df[(df["{}_hypothesis".format(key)] == 1) & (df['label'] == golden_label)])
    count_true_negative = len(df[(df["{}_hypothesis".format(key)] == 0) & (df['label'] != golden_label)])
    count_false_positive = len(df[(df["{}_hypothesis".format(key)] == 1) & (df['label'] != golden_label)])
    count_false_negative = len(df[(df["{}_hypothesis".format(key)] == 0) & (df['label'] == golden_label)])

    print("precision = {}".format(round(0 if count_true_positive == 0 else count_true_positive / (count_true_positive + count_false_positive), 4)))
    print("recall = {}".format(round(count_true_positive / (count_true_positive + count_false_negative), 4)))
    print("accuracy = {}".format(round((count_true_positive + count_true_negative) / (count_true_positive + count_true_negative + count_false_positive + count_false_negative), 4)))

    real_precision = []
    estimate_precision = []
    real_recall = []
    estimate_recall = []
    real_accuracy = []
    estimate_accuracy = []
    for prompt_id in range(10):
        real_count_true_positive = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 1) & (df['label'] == golden_label)])
        real_count_true_negative = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 0) & (df['label'] != golden_label)])
        real_count_false_positive = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 1) & (df['label'] != golden_label)])
        real_count_false_negative = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 0) & (df['label'] == golden_label)])
        # print("precision = {}".format(round(real_count_true_positive / (real_count_true_positive + real_count_false_positive), 4)))
        # print("recall = {}".format(round(real_count_true_positive / (real_count_true_positive + real_count_false_negative), 4)))
        # print("accuracy = {}".format(round((real_count_true_positive + real_count_true_negative) / (real_count_true_positive + real_count_true_negative + real_count_false_positive + real_count_false_negative), 4)))
        # print("prompt {prompt_id}".format(prompt_id=prompt_id), round(real_count_true_positive / (real_count_true_positive + real_count_false_positive), 4))
        real_precision.append(round(real_count_true_positive / (real_count_true_positive + real_count_false_positive), 4))
        real_recall.append(round(real_count_true_positive / (real_count_true_positive + real_count_false_negative), 4))
        real_accuracy.append(round((real_count_true_positive + real_count_true_negative) / (real_count_true_positive + real_count_true_negative + real_count_false_positive + real_count_false_negative), 4))
        estimate_count_true_positive = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 1) & (df["{}_hypothesis".format(key)] == 1)])
        estimate_count_true_negative = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 0) & (df["{}_hypothesis".format(key)] == 0)])
        estimate_count_false_positive = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 1) & (df["{}_hypothesis".format(key)] == 0)])
        estimate_count_false_negative = len(df[(df['{key}_prompt{id}'.format(key=key,id=prompt_id)] == 0) & (df["{}_hypothesis".format(key)] == 1)])
        # print("prompt {prompt_id}".format(prompt_id=prompt_id), round(estimate_count_true_positive / (estimate_count_true_positive + estimate_count_false_positive), 4))
        estimate_precision.append(round(estimate_count_true_positive / (estimate_count_true_positive + estimate_count_false_positive), 4))
        estimate_recall.append(0 if estimate_count_true_positive == 0 else round(estimate_count_true_positive / (estimate_count_true_positive + estimate_count_false_negative), 4))
        estimate_accuracy.append(round((estimate_count_true_positive + estimate_count_true_negative) / (estimate_count_true_positive + estimate_count_true_negative + estimate_count_false_positive + estimate_count_false_negative), 4))
    print("max real precision: {}".format(max(real_precision)))
    print("min real precision: {}".format(min(real_precision)))
    print("max real recall: {}".format(max(real_recall)))
    print("min real recall: {}".format(min(real_recall)))
    print("max real accuracy: {}".format(max(real_accuracy)))
    print("min real accuracy: {}".format(min(real_accuracy)))

    index_max_estimate_precision = estimate_precision.index(max(estimate_precision))
    print("max estimate precision: {}".format(real_precision[index_max_estimate_precision]))
    index_max_estimate_accuracy = estimate_accuracy.index(max(estimate_accuracy))
    print("max estimate accuracy: {}".format(real_accuracy[index_max_estimate_accuracy]))
    print("real precision: {}".format(real_precision))
    print("real recall: {}".format(real_recall))
    # print("estimate precision: {}".format(estimate_precision))
    print("real accuracy: {}".format(real_accuracy))
    # print("estimate accuracy: {}".format(estimate_accuracy))
    for item in real_precision:
        print(item)
    print("----")
    for item in real_recall:
        print(item)
    print("----")
    for item in real_accuracy:
        print(item)
        
    
test_df = df
# test_df = df.sample(n=300)
for threshold in [0.8]:
    print("threshold: {threshold}".format(threshold=threshold))
    analyze(test_df, threshold, key, golden_label)
    print("--------------------------------------------------")


threshold: 0.8
precision = 0.5
recall = 0.1744
accuracy = 0.785
max real precision: 0.4773
min real precision: 0.23
max real recall: 0.7326
min real recall: 0.0814
max real accuracy: 0.775
min real accuracy: 0.5825
max estimate precision: 0.3333
max estimate accuracy: 0.7675
real precision: [0.2324, 0.3378, 0.2834, 0.3539, 0.2349, 0.2874, 0.3067, 0.3333, 0.4773, 0.23]
real recall: [0.3837, 0.5814, 0.6163, 0.7326, 0.407, 0.5581, 0.5814, 0.0814, 0.4884, 0.2674]
real accuracy: [0.595, 0.665, 0.5825, 0.655, 0.5875, 0.6075, 0.6275, 0.7675, 0.775, 0.65]
0.2324
0.3378
0.2834
0.3539
0.2349
0.2874
0.3067
0.3333
0.4773
0.23
----
0.3837
0.5814
0.6163
0.7326
0.407
0.5581
0.5814
0.0814
0.4884
0.2674
----
0.595
0.665
0.5825
0.655
0.5875
0.6075
0.6275
0.7675
0.775
0.65
--------------------------------------------------


In [198]:
import numpy as np
m = 500
n = 10
# test_df = df
m = test_df.shape[0]
X = np.array([[1 if row["{key}_prompt{id}".format(key=key, id=i)] == 1 else -1 for i in range(10)] for index, row in test_df.iterrows()])
print(X.shape)

(400, 10)


In [199]:
# SVD to find first singular vector
from sklearn.utils.extmath import randomized_svd
# U, s, Vt = np.linalg.svd(X, full_matrices=False)
# print(Vt)
U, s, Vt = randomized_svd(X, n_components=1)
print(Vt)
Vt = Vt * s[0] * np.sign(Vt[0][0])
w_hat = Vt / np.sqrt(m)
w_tilde = w_hat
w_tilde[w_tilde > 1] = 1
w_tilde[w_tilde < -1] = -1

q_hat = np.sign(np.matmul(X, w_tilde[0]))

print(q_hat.shape)

test_df["estimate_result"] = q_hat

count_true_positive = len(test_df[(test_df['estimate_result'] == 1) & (test_df['label'] == golden_label)])
count_true_negative = len(test_df[(test_df['estimate_result'] == -1) & (test_df['label'] != golden_label)])
count_false_positive = len(test_df[(test_df['estimate_result'] == 1) & (test_df['label'] != golden_label)])
count_false_negative = len(test_df[(test_df['estimate_result'] == -1) & (test_df['label'] == golden_label)])
print("true positive: {count}".format(count=count_true_positive))
print("true negative: {count}".format(count=count_true_negative))
print("false positive: {count}".format(count=count_false_positive))
print("false negative: {count}".format(count=count_false_negative))

print("precision: {precision}".format(precision=count_true_positive / (count_true_positive + count_false_positive)))
print("recall: {recall}".format(recall=count_true_positive / (count_true_positive + count_false_negative)))
print("accuracy: {accuracy}".format(accuracy=(count_true_positive + count_true_negative) / len(test_df)))

print(w_hat)
for item in w_hat[0]:
    print(item)


[[-0.1993306  -0.37854607 -0.33478588 -0.3339902  -0.15481116 -0.37139027
  -0.3512498  -0.30495271 -0.39362548 -0.24521079]]
(400,)
true positive: 48
true negative: 237
false positive: 77
false negative: 38
precision: 0.384
recall: 0.5581395348837209
accuracy: 0.7125
[[0.39075279 0.74207337 0.65628918 0.65472938 0.3034802  0.72804567
  0.6885638  0.59780647 0.77163392 0.48069287]]


# Cubam Analysis

In [200]:
output = [[len(df), prompt_num, len(df) * prompt_num]]
i = 0
for index, row in df.iterrows():
    for prompt_id in range(prompt_num):
        output.append([i, prompt_id, row["{key}_prompt{num}".format(key=key, num=prompt_id)]])
    i += 1
with open("label.txt", 'w') as f:
    for line in output:
        f.write(" ".join([str(i) for i in line]) + "\n")

In [201]:
from cubam.Binary1dSignalModel import Binary1dSignalModel
model = Binary1dSignalModel()
model.load_data("label.txt")
model.optimize_param()
eprm = {
    'wkr' : model.get_worker_param(), 
    'img' : model.get_image_param()
}
score = []
w = 2
for index in range(len(eprm['wkr'])):
    print(index, eprm['wkr'][index], eprm['wkr'][index][1] / eprm['wkr'][index][0], 1 / eprm['wkr'][index][0])
    score.append(round(8 - abs(8 - eprm['wkr'][index][1] / eprm['wkr'][index][0]) + w * eprm['wkr'][index][0], 4))
    
for index in range(len(eprm['wkr'])):
    print(eprm['wkr'][index][1] / eprm['wkr'][index][0])
print("----")
for index in range(len(eprm['wkr'])):
    print(1 / eprm['wkr'][index][0])
print("score")
for index in range(len(score)):
    print(index, score[index])
getParameter = lambda prmdict, pidx: [prmdict[i][pidx] for i \
                                          in range(len(prmdict))]
exi = getParameter(model.get_image_param(), 0)
print(exi)
estimated_result_binary_biased = [1 if item > 0 else 0 for item in exi]
print(estimated_result_binary_biased)

0 [0.4446456096171754, 0.21379937526649273] 0.48083095985264906 2.248982062053791
1 [3.172758533777183, -0.5641261344362055] -0.17780304691659307 0.3151831409021523
2 [3.436186205080825, -1.1998247219100042] -0.34917337137781285 0.29102031738599515
3 [2.7086938679319257, -0.8375718435492641] -0.3092161330836349 0.369181623600564
4 [0.29179406410424463, 0.21817924256140594] 0.7477165213458917 3.4270745125327355
5 [3.959005306353944, -1.0554132712806252] -0.2665854651891363 0.2525886990843547
6 [2.674907186779974, -0.6516298888518697] -0.24360841081603848 0.37384474681672597
7 [1.5049356146651278, 1.7496457403099246] 1.1626050465283517 0.6644802543413234
8 [2.6871688966755047, 0.4051346015291729] 0.1507663333072941 0.37213887122509265
9 [0.44800648165338014, 0.5286294900685734] 1.1799594687060149 2.2321105630201434
0.48083095985264906
-0.17780304691659307
-0.34917337137781285
-0.3092161330836349
0.7477165213458917
-0.2665854651891363
-0.24360841081603848
1.1626050465283517
0.150766333307

In [202]:
from cubam.BinaryBiasModel import BinaryBiasModel
model = BinaryBiasModel()
model.load_data("label.txt")
model.optimize_param()
eprm = {
    'wkr' : model.get_worker_param(), 
    'img' : model.get_image_param()
}
score = []
w = 2
for index in range(len(eprm['wkr'])):
    print(index, eprm['wkr'][index], eprm['wkr'][index][1] / eprm['wkr'][index][0], 1 / eprm['wkr'][index][0])
    score.append(round(2 - abs(2 - eprm['wkr'][index][1] / eprm['wkr'][index][0]) + w * eprm['wkr'][index][0], 4))
print("score")
for index in range(len(score)):
    print(index, score[index])
getParameter = lambda prmdict, pidx: [prmdict[i][pidx] for i \
                                          in range(len(prmdict))]
exi = getParameter(model.get_image_param(), 0)
print(exi)
estimated_result_binary_biased = [1 if item > 0.5 else 0 for item in exi]
print(estimated_result_binary_biased)

0 [0.36363636 0.65656566] 1.805555555334028 2.7499999997937503
1 [0.78787879 0.84848485] 1.0769230769133136 1.2692307693235207
2 [0.91919192 0.77777778] 0.8461538461705833 1.0879120880113151
3 [0.87878788 0.78787879] 0.8965517241497027 1.1379310345808562
4 [0.38383838 0.63636364] 1.6578947366707062 2.60526315773705
5 [0.86868687 0.82828283] 0.9534883720983776 1.1511627907953894
6 [0.84848485 0.82828283] 0.9761904761932824 1.1785714286682398
7 [0.06060606 0.95959596] 15.833333308858334 16.499999976075003
8 [0.5959596  0.96969697] 1.627118643962568 1.6779661017489516
9 [0.26262626 0.75757576] 2.8846153838977813 3.8076923070039945
score
0 2.5328
1 2.6527
2 2.6845
3 2.6541
4 2.4256
5 2.6909
6 2.6732
7 -11.7121
8 2.819
9 1.6406
[0.9999338450748534, 0.9999123117222124, 0.9207285896071209, 0.22945303084376734, 1.2569089959732664e-05, 0.016120586212857498, 0.8833287105238407, 1.2569089959732664e-05, 0.01333676945419096, 0.00026137202380279047, 1.2569089959732664e-05, 0.9962252969109889, 1.2569

In [203]:
df["estimate_result"] = estimated_result_binary_biased

count_true_positive = len(df[(df['estimate_result'] == 1) & (df['label'] == golden_label)])
count_true_negative = len(df[(df['estimate_result'] == 0) & (df['label'] != golden_label)])
count_false_positive = len(df[(df['estimate_result'] == 1) & (df['label'] != golden_label)])
count_false_negative = len(df[(df['estimate_result'] == 0) & (df['label'] == golden_label)])
print("true positive: {count}".format(count=count_true_positive))
print("true negative: {count}".format(count=count_true_negative))
print("false positive: {count}".format(count=count_false_positive))
print("false negative: {count}".format(count=count_false_negative))

print("precision: {precision}".format(precision=count_true_positive / (count_true_positive + count_false_positive)))
print("recall: {recall}".format(recall=count_true_positive / (count_true_positive + count_false_negative)))
print("accuracy: {accuracy}".format(accuracy=(count_true_positive + count_true_negative) / len(df)))

true positive: 50
true negative: 237
false positive: 77
false negative: 36
precision: 0.3937007874015748
recall: 0.5813953488372093
accuracy: 0.7175
